<a href="https://colab.research.google.com/github/Parv17k/nyc_collision_data_analysis/blob/main/data_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [3]:
!wget -q https://downloads.apache.org/spark/spark-3.1.1/spark-3.1.1-bin-hadoop2.7.tgz

In [4]:
!wget -O longlat.csv -q https://public.opendatasoft.com/explore/dataset/us-zip-code-latitude-and-longitude/download/?format=csv&timezone=America/New_York&lang=en&use_labels_for_header=true&csv_separator=%3B

In [5]:
ls

longlat.csv  sample_data/  spark-3.1.1-bin-hadoop2.7.tgz


In [6]:
!tar xf spark-3.1.1-bin-hadoop2.7.tgz

In [7]:
!pip install -q findspark

In [8]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop2.7"

In [9]:
import findspark
findspark.init()

In [10]:
import findspark
findspark.init()

In [11]:
findspark.find()

'/content/spark-3.1.1-bin-hadoop2.7'

In [12]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Collision")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [13]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -y ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels

--2021-03-29 00:59:48--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 3.209.27.98, 54.209.152.48, 3.224.116.172, ...
Connecting to bin.equinox.io (bin.equinox.io)|3.209.27.98|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13805791 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.17M  36.7MB/s    in 0.4s    

2021-03-29 00:59:49 (36.7 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13805791/13805791]

UnZip 6.00 of 20 April 2009, by Debian. Original by Info-ZIP.

Usage: unzip [-Z] [-opts[modifiers]] file[.zip] [list] [-x xlist] [-d exdir]
  Default action is to extract files in list, except those in xlist, to exdir;
  file[.zip] may be a wildcard.  -Z => ZipInfo mode ("unzip -Z" for usage).

  -p  extract files to pipe, no messages     -l  list files (short format)
  -f  freshen existing files, create none    -t  test co

In [14]:
!wget -q https://data.cityofnewyork.us/api/views/h9gi-nx95/rows.csv

In [15]:
mv rows.csv col.csv


In [16]:
ls


col.csv      ngrok-stable-linux-amd64.zip  spark-3.1.1-bin-hadoop2.7/
longlat.csv  sample_data/                  spark-3.1.1-bin-hadoop2.7.tgz


In [17]:
from pyspark import SparkContext,SparkConf
from pyspark.sql import SparkSession
import pyspark
conf = pyspark.SparkConf()
conf.set("spark.driver.memory","8G");
import numpy as np
sc = SparkContext.getOrCreate(conf=conf)
from pyspark.sql.functions import round, col


In [18]:
spark =SparkSession(sc)

In [19]:
col=spark.read.option("header",True).csv("./col.csv",inferSchema=True)
longlat=spark.read.option("delimiter",";").csv("./longlat.csv",inferSchema=True,header=True)

In [20]:
col.count()-col.distinct().count()

0

In [21]:
len(col.columns)


29

In [22]:
col.printSchema()

root
 |-- CRASH DATE: string (nullable = true)
 |-- CRASH TIME: string (nullable = true)
 |-- BOROUGH: string (nullable = true)
 |-- ZIP CODE: string (nullable = true)
 |-- LATITUDE: double (nullable = true)
 |-- LONGITUDE: double (nullable = true)
 |-- LOCATION: string (nullable = true)
 |-- ON STREET NAME: string (nullable = true)
 |-- CROSS STREET NAME: string (nullable = true)
 |-- OFF STREET NAME: string (nullable = true)
 |-- NUMBER OF PERSONS INJURED: string (nullable = true)
 |-- NUMBER OF PERSONS KILLED: integer (nullable = true)
 |-- NUMBER OF PEDESTRIANS INJURED: integer (nullable = true)
 |-- NUMBER OF PEDESTRIANS KILLED: integer (nullable = true)
 |-- NUMBER OF CYCLIST INJURED: integer (nullable = true)
 |-- NUMBER OF CYCLIST KILLED: string (nullable = true)
 |-- NUMBER OF MOTORIST INJURED: string (nullable = true)
 |-- NUMBER OF MOTORIST KILLED: integer (nullable = true)
 |-- CONTRIBUTING FACTOR VEHICLE 1: string (nullable = true)
 |-- CONTRIBUTING FACTOR VEHICLE 2: strin

In [23]:
from pyspark.sql.types import IntegerType
col = col.withColumn("NUMBER OF PERSONS INJURED", col["NUMBER OF PERSONS INJURED"].cast(IntegerType()))
col=col.withColumn("NUMBER OF CYCLIST KILLED", col["NUMBER OF CYCLIST KILLED"].cast(IntegerType()))
col=col.withColumn("NUMBER OF MOTORIST INJURED", col["NUMBER OF MOTORIST INJURED"].cast(IntegerType()))

In [24]:
col.printSchema()

root
 |-- CRASH DATE: string (nullable = true)
 |-- CRASH TIME: string (nullable = true)
 |-- BOROUGH: string (nullable = true)
 |-- ZIP CODE: string (nullable = true)
 |-- LATITUDE: double (nullable = true)
 |-- LONGITUDE: double (nullable = true)
 |-- LOCATION: string (nullable = true)
 |-- ON STREET NAME: string (nullable = true)
 |-- CROSS STREET NAME: string (nullable = true)
 |-- OFF STREET NAME: string (nullable = true)
 |-- NUMBER OF PERSONS INJURED: integer (nullable = true)
 |-- NUMBER OF PERSONS KILLED: integer (nullable = true)
 |-- NUMBER OF PEDESTRIANS INJURED: integer (nullable = true)
 |-- NUMBER OF PEDESTRIANS KILLED: integer (nullable = true)
 |-- NUMBER OF CYCLIST INJURED: integer (nullable = true)
 |-- NUMBER OF CYCLIST KILLED: integer (nullable = true)
 |-- NUMBER OF MOTORIST INJURED: integer (nullable = true)
 |-- NUMBER OF MOTORIST KILLED: integer (nullable = true)
 |-- CONTRIBUTING FACTOR VEHICLE 1: string (nullable = true)
 |-- CONTRIBUTING FACTOR VEHICLE 2: st

In [25]:
import seaborn as sns

In [26]:
#dropping columns with collision id = null

In [27]:
col.filter(col.COLLISION_ID.isNull()).drop()

DataFrame[CRASH DATE: string, CRASH TIME: string, BOROUGH: string, ZIP CODE: string, LATITUDE: double, LONGITUDE: double, LOCATION: string, ON STREET NAME: string, CROSS STREET NAME: string, OFF STREET NAME: string, NUMBER OF PERSONS INJURED: int, NUMBER OF PERSONS KILLED: int, NUMBER OF PEDESTRIANS INJURED: int, NUMBER OF PEDESTRIANS KILLED: int, NUMBER OF CYCLIST INJURED: int, NUMBER OF CYCLIST KILLED: int, NUMBER OF MOTORIST INJURED: int, NUMBER OF MOTORIST KILLED: int, CONTRIBUTING FACTOR VEHICLE 1: string, CONTRIBUTING FACTOR VEHICLE 2: string, CONTRIBUTING FACTOR VEHICLE 3: string, CONTRIBUTING FACTOR VEHICLE 4: string, CONTRIBUTING FACTOR VEHICLE 5: string, COLLISION_ID: int, VEHICLE TYPE CODE 1: string, VEHICLE TYPE CODE 2: string, VEHICLE TYPE CODE 3: string, VEHICLE TYPE CODE 4: string, VEHICLE TYPE CODE 5: string]

In [28]:
col.count()

1764791

In [29]:
missingLocation=col.filter(col["BOROUGH"].isNull()).filter(col["ZIP CODE"].isNull())

In [30]:
cannotRetriveLocation =missingLocation.filter(col["LONGITUDE"].isNull())

In [31]:
cannotRetriveLocation.count()

177335

In [32]:
# Infer the schema, and register the DataFrame as a table.

col.createOrReplaceTempView("col");
cannotRetriveLocation.createOrReplaceTempView("toDelete")

In [33]:
colNew= spark.sql("select * FROM col WHERE col.COLLISION_ID not in (select COLLISION_ID from toDelete where COLLISION_ID is not null)");


In [34]:
colNew.count()

1587454

In [35]:
col.count() - cannotRetriveLocation.count()

1587456

In [36]:
colNew.filter(colNew["ZIP CODE"].isNull()).filter(colNew["BOROUGH"].isNotNull()).count()

211

In [37]:
BOROUGH=list(map(lambda x : x["BOROUGH"],colNew.select("BOROUGH").distinct().collect()))

In [38]:
BOROUGH

[None, 'QUEENS', 'BROOKLYN', 'BRONX', 'MANHATTAN', 'STATEN ISLAND']

In [39]:
longlat.printSchema()

root
 |-- zip: integer (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- timezone: integer (nullable = true)
 |-- dst: integer (nullable = true)
 |-- geopoint: string (nullable = true)



In [40]:

colNew.createOrReplaceTempView("colNew");
longlat.createOrReplaceTempView("longlat");

In [41]:
missingZipCode=colNew.filter(colNew['ZIP CODE'].isNull());


In [42]:
missingZipCode.createOrReplaceTempView("missingZipCode");


In [43]:
"""""""""""
for m in missingZipCode.limit(19).collect():
  zip=spark.sql("select zip,city FROM longlat where ROUND(longlat.latitude,2) ="+str(round(m.LATITUDE,2))+" and ROUND(longlat.longitude,2)= "+str(round(m.LONGITUDE,2))+" limit 1");
  zip=zip.select("zip","city").collect()
  if len(zip)>0:
    if zip[0].city.upper()=="NEW YORK":
      print(m["COLLISION_ID"],"MANHATTAN",zip[0].zip)
    else:
        if zip[0].city.upper() in BOROUGH:
          print(m["COLLISION_ID"],zip[0].city.upper(),zip[0].zip)
          """"""


SyntaxError: ignored

In [44]:
colNew.select("NUMBER OF CYCLIST INJURED").distinct().limit(100).show();

+-------------------------+
|NUMBER OF CYCLIST INJURED|
+-------------------------+
|                        1|
|                        3|
|                        4|
|                        2|
|                        0|
+-------------------------+



NUMBER OF CYCLIST INJURED Is cleaned.

In [45]:
print(colNew.select("NUMBER OF PERSONS INJURED").distinct().show());

+-------------------------+
|NUMBER OF PERSONS INJURED|
+-------------------------+
|                       31|
|                       27|
|                       12|
|                       22|
|                     null|
|                        1|
|                       13|
|                        6|
|                       16|
|                        3|
|                       20|
|                        5|
|                       19|
|                       15|
|                       43|
|                        9|
|                       17|
|                        4|
|                        8|
|                        7|
+-------------------------+
only showing top 20 rows

None


NUMBER OF PERSONS INJURED has Null value hence making it to Zero.

In [ ]:
colNew=colNew.fillna(value=0,subset=["NUMBER OF PERSONS INJURED"])

In [ ]:
colNew

In [46]:
#NUMBER OF PERSONS KILLED
print(colNew.select("NUMBER OF PERSONS KILLED").distinct().show());

+------------------------+
|NUMBER OF PERSONS KILLED|
+------------------------+
|                    null|
|                       1|
|                       3|
|                       5|
|                       4|
|                       8|
|                       2|
|                       0|
+------------------------+

None


NUMBER OF PERSONS KILLED has NULL values : replacign them with 0.


In [ ]:
colNew=colNew.fillna(value=0,subset=["NUMBER OF PERSONS KILLED"])

In [ ]:
print(colNew.select("NUMBER OF PEDESTRIANS INJURED").distinct().show());

NUMBER OF PEDESTRIANS INJURED has no null value

In [ ]:
#NUMBER OF PEDESTRIANS KILLED
print(colNew.select("NUMBER OF PEDESTRIANS KILLED").distinct().show());



```
NUMBER OF PEDESTRIANS KILLED has no null value
```



In [ ]:
print(colNew.select("NUMBER OF CYCLIST INJURED").distinct().show());

NUMBER OF CYCLIST INJURED has no null value

In [ ]:
print(colNew.select("NUMBER OF CYCLIST KILLED").distinct().show());

NUMBER OF CYCLIST KILLED has no null value

In [ ]:
#NUMBER OF MOTORIST INJURED
print(colNew.select("NUMBER OF MOTORIST INJURED").distinct().show(100));

NUMBER OF MOTORIST INJURED has no null value

In [ ]:
#NUMBER OF MOTORIST INJURED
print(colNew.select("NUMBER OF MOTORIST KILLED").distinct().show(100));

NUMBER OF MOTORIST INJURED has no null value

In [48]:
#CONTRIBUTING FACTOR VEHICLE
print(colNew.select("CONTRIBUTING FACTOR VEHICLE 1").distinct().show(70));

# has numerical values 1 and 80
# has null value.


+-----------------------------+
|CONTRIBUTING FACTOR VEHICLE 1|
+-----------------------------+
|         Following Too Clo...|
|         Traffic Control D...|
|         Driverless/Runawa...|
|         Accelerator Defec...|
|         Windshield Inadeq...|
|         Using On Board Na...|
|                 Unsafe Speed|
|         Shoulders Defecti...|
|               Tinted Windows|
|            Oversized Vehicle|
|         Passing or Lane U...|
|         Lane Marking Impr...|
|         Other Lighting De...|
|                         null|
|         Aggressive Drivin...|
|              Other Vehicular|
|          Driver Inexperience|
|          Tow Hitch Defective|
|                      Texting|
|                       Illnes|
|              Drugs (illegal)|
|                  Unspecified|
|           Pavement Defective|
|         Prescription Medi...|
|         Cell Phone (hand-...|
|         View Obstructed/L...|
|           Lost Consciousness|
|         Reaction to Other...|
|       

In [47]:
#CONTRIBUTING FACTOR VEHICLE
print(colNew.select("CONTRIBUTING FACTOR VEHICLE 2").distinct().show(100));

# has numerical values 1 and 80.
# has null value.


+-----------------------------+
|CONTRIBUTING FACTOR VEHICLE 2|
+-----------------------------+
|         Following Too Clo...|
|         Traffic Control D...|
|         Driverless/Runawa...|
|         Accelerator Defec...|
|         Windshield Inadeq...|
|         Using On Board Na...|
|                 Unsafe Speed|
|         Shoulders Defecti...|
|               Tinted Windows|
|            Oversized Vehicle|
|         Passing or Lane U...|
|         Lane Marking Impr...|
|         Other Lighting De...|
|                         null|
|         Aggressive Drivin...|
|              Other Vehicular|
|          Driver Inexperience|
|          Tow Hitch Defective|
|                      Texting|
|                       Illnes|
|              Drugs (illegal)|
|                  Unspecified|
|           Pavement Defective|
|         Prescription Medi...|
|         Cell Phone (hand-...|
|         View Obstructed/L...|
|           Lost Consciousness|
|         Reaction to Other...|
|       

In [ ]:
#CONTRIBUTING FACTOR VEHICLE
print(colNew.select("CONTRIBUTING FACTOR VEHICLE 3").distinct().show(100));

# has numerical values 1 and 80.
# has null value.

In [49]:
#CONTRIBUTING FACTOR VEHICLE
print(colNew.select("CONTRIBUTING FACTOR VEHICLE 4").distinct().show(100));
# has null value.

+-----------------------------+
|CONTRIBUTING FACTOR VEHICLE 4|
+-----------------------------+
|         Following Too Clo...|
|         Traffic Control D...|
|         Accelerator Defec...|
|         Driverless/Runawa...|
|         Windshield Inadeq...|
|                 Unsafe Speed|
|         Passing or Lane U...|
|                         null|
|         Aggressive Drivin...|
|              Other Vehicular|
|          Driver Inexperience|
|              Drugs (illegal)|
|                  Unspecified|
|           Pavement Defective|
|         Prescription Medi...|
|         View Obstructed/L...|
|           Lost Consciousness|
|         Reaction to Uninv...|
|                  Fell Asleep|
|         Outside Car Distr...|
|              Drugs (Illegal)|
|              Fatigued/Drowsy|
|         Driver Inattentio...|
|         Failure to Keep R...|
|           Obstruction/Debris|
|         Unsafe Lane Changing|
|          Alcohol Involvement|
|                      Illness|
|       

In [52]:
print(colNew.select("CONTRIBUTING FACTOR VEHICLE 5").distinct().show(30));

+-----------------------------+
|CONTRIBUTING FACTOR VEHICLE 5|
+-----------------------------+
|         Following Too Clo...|
|         Traffic Control D...|
|                 Unsafe Speed|
|         Passing or Lane U...|
|                         null|
|         Aggressive Drivin...|
|              Other Vehicular|
|          Driver Inexperience|
|              Drugs (illegal)|
|                  Unspecified|
|           Pavement Defective|
|         Reaction to Uninv...|
|                  Fell Asleep|
|         Tire Failure/Inad...|
|         Outside Car Distr...|
|              Fatigued/Drowsy|
|         Driver Inattentio...|
|         Failure to Keep R...|
|           Obstruction/Debris|
|          Alcohol Involvement|
|                      Illness|
|         Traffic Control D...|
|             Brakes Defective|
|             Backing Unsafely|
|         Failure to Yield ...|
|                        Glare|
|             Steering Failure|
|          Passing Too Closely|
|       

# ***TODO***
Treat Null Values for all column to  Unknown. 1,2,3,4,5


Treat **1 and 80** for CONTRIBUTING FACTOR VEHICLE 1,2,3

In [ ]:
# zip,borough,location,long,lat, [contributing1,2,3,4], killed and injured [3-4],COLLISION_ID.